In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
%matplotlib inline
# import _init_paths
from PIL import Image
import matplotlib.pyplot as plt
# from ult.config import cfg
import pickle
import json
import numpy as np
import cv2
import os
import ipdb
import torch
import scipy.misc as miscp
import _init_paths
from core.config import cfg, merge_cfg_from_file, merge_cfg_from_list, assert_and_infer_cfg
from core.test_engine import run_inference, get_inference_dataset, extend_results
from datasets.json_dataset import JsonDataset

/root/backup/Research/HOI_plus/lib/utils/env.py:44: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/root/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/root/anaconda3/lib/python3

/root/backup/Research/HOI_plus/tools ..coco evaling


/root/backup/Research/HOI_plus/lib/utils/vis.py:36: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/root/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/root/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/root/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/root/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/root/anaconda3/lib/python3

In [3]:
cfg.TEST.DATASETS = ('vcoco_test',)
cfg.MODEL.NUM_CLASSES = 81
cfg.TEST.PRECOMPUTED_PROPOSALS = False
cfg.MODEL.VCOCO_ON = True
cfg.VCOCO.USE_PRECOMP_BOX = True
dataset_name, proposal_file = get_inference_dataset(0, is_parent=False)
json_dataset = JsonDataset(dataset_name)
vcocodb = json_dataset.get_roidb(gt=True)

loading annotations into memory...
original image numer: 4946
original annotation number: 42999
Done (t=1.50s)
creating index...
index created!
final image number: 4946
final anns number: 42999
loading vcoco annotations...
add precomp box from /root/backup/Research/HOI_plus/data/cache/vcoco_test_precomp_boxes_ican.json


In [4]:
det_name = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test_0.5_0.4_all/hoi_vcoco_test_results.pkl'
all_hoi = pickle.load(open(det_name, 'rb') , encoding='latin1')
# image_ids = list(all_hoi.keys())

FileNotFoundError: [Errno 2] No such file or directory: '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test_0.5_0.4_all/hoi_vcoco_test_results.pkl'

In [30]:
json_dataset.VCOCO._do_role_eval(vcocodb, all_hoi, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 38.80 (#pos = 3608)
              sit-instr: AP = 24.93 (#pos = 1916)
             ride-instr: AP = 64.06 (#pos = 556)
               look-obj: AP = 36.09 (#pos = 3347)
              hit-instr: AP = 71.91 (#pos = 349)
                hit-obj: AP = 45.02 (#pos = 349)
                eat-obj: AP = 38.99 (#pos = 521)
              eat-instr: AP = 4.13 (#pos = 521)
             jump-instr: AP = 47.85 (#pos = 635)
              lay-instr: AP = 23.80 (#pos = 387)
    talk_on_phone-instr: AP = 52.28 (#pos = 285)
              carry-obj: AP = 34.45 (#pos = 472)
              throw-obj: AP = 44.35 (#pos = 244)
              catch-obj: AP = 46.01 (#pos = 246)
              cut-instr: AP = 34.01 (#pos = 269)
                cut-obj: AP = 37.70 (#pos = 269)
 work_on_computer-instr: AP = 58.67 (#pos = 410)
              ski-instr: AP = 46.58 (#pos = 424)
             surf-instr: AP = 78.60 (#pos = 486)
       skateboard-

In [5]:
def get_overlap(boxes, ref_box):
    ixmin = np.maximum(boxes[:, 0], ref_box[0])
    iymin = np.maximum(boxes[:, 1], ref_box[1])
    ixmax = np.minimum(boxes[:, 2], ref_box[2])
    iymax = np.minimum(boxes[:, 3], ref_box[3])
    # maximum zero
    iw = np.maximum(ixmax - ixmin + 1., 0.)
    ih = np.maximum(iymax - iymin + 1., 0.)
    inters = iw * ih

    # union
    uni = ((ref_box[2] - ref_box[0] + 1.) * (ref_box[3] - ref_box[1] + 1.) +
           (boxes[:, 2] - boxes[:, 0] + 1.) *
           (boxes[:, 3] - boxes[:, 1] + 1.) - inters)

    overlaps = inters / uni
    return overlaps

In [6]:
count = 0
num_actions = json_dataset.VCOCO.num_actions
roles = json_dataset.VCOCO.roles
fp1 = fp2 = fp3 = fp4 = fp5 = fp6 = fp7 = tp = 0
all_res = []
set_zero_ret = {}

for i in range(len(vcocodb)):
    res = []
    image_id = vcocodb[i]['id']

    pred_roles = all_hoi[image_id]['roles']
    pred_agents = all_hoi[image_id]['agents']
    gt_human_inds, gt_act_inds, gt_r_ids = np.where(vcocodb[i]['gt_role_id'] > -1)
    gt_obj_inds = vcocodb[i]['gt_role_id'][gt_human_inds, gt_act_inds, gt_r_ids]
    if len(gt_obj_inds) == 0:
#         set_zero_ret[image_id] = {'agents': pred_agents, 'roles': pred_roles}
        continue
    gt_all_relboxes = vcocodb[i]['boxes'][np.unique(gt_obj_inds)]
        
    gt_h_inds = np.where(vcocodb[i]['gt_classes'] == 1)[0]
    gt_h_boxes = vcocodb[i]['boxes'][gt_h_inds]
    gt_actions = vcocodb[i]['gt_actions'][gt_h_inds]
    ignore = np.any(gt_actions == -1, axis=1)
    
    for aid in range(num_actions):
        if len(roles[aid]) < 2:
            continue

        for rid in range(len(roles[aid]) - 1):
            
            covered = np.zeros((gt_h_boxes.shape[0]), dtype=np.bool)
            gt_role_inds = vcocodb[i]['gt_role_id'][gt_h_inds, aid, rid]
            gt_roles = -np.ones_like(gt_h_boxes)
            for j in range(gt_h_boxes.shape[0]):
                if gt_role_inds[j] > -1:
                    gt_roles[j] = vcocodb[i]['boxes'][gt_role_inds[j]]
            
            agent_boxes = pred_agents[:, :4]
            role_boxes = pred_roles[:, 5 * aid: 5 * aid + 4, rid]
            agent_scores = pred_roles[:, 5 * aid + 4, rid]
            
            if role_boxes.shape[0] == 0: 
                continue
                
            valid = np.where(np.isnan(agent_scores) == False)[0]
            agent_scores = agent_scores[valid]
            agent_boxes = agent_boxes[valid, :]
            role_boxes = role_boxes[valid, :]
            
            idx = agent_scores.argsort()[::-1]
            for j in idx:  # in this image, this action with highest action score
                pred_box = agent_boxes[j, :]
                overlaps = get_overlap(gt_h_boxes, pred_box)  # gt_h_boxes: gt human box

                jmax = overlaps.argmax()  # which gt_box best matches this detected box
                ovmax = overlaps.max()

                if ignore[jmax]:
                    continue
                count += 1
                
                ov_all_relboxes = get_overlap(gt_all_relboxes, role_boxes[j, :])
                ov_all_boxes = get_overlap(vcocodb[i]['boxes'], role_boxes[j, :])
                ov_max_all_relboxes = ov_all_relboxes.max()
                ov_max_all_boxes = ov_all_boxes.max()

                gt_role_j_human = vcocodb[i]['gt_role_id'][gt_h_inds[jmax]]
                gt_box_j_objs = vcocodb[i]['boxes'][np.unique(gt_role_j_human[np.where(gt_role_j_human > -1)])]
                if len(gt_box_j_objs) == 0:
                    ov_max_j_objs = -1
                else:
                    ov_j_objs = get_overlap(gt_box_j_objs, role_boxes[j, :])
                    ov_max_j_objs = ov_j_objs.max()
                    
                if np.all(gt_roles[jmax, :] == -1):
                    if np.all(role_boxes[j, :] == 0.0) or np.all(np.isnan(role_boxes[j, :])):
                        ov_role = 1.0
                    else:
                        ov_role = -1.0
                else:
                    ov_role = get_overlap(gt_roles[jmax, :].reshape((1, 4)), role_boxes[j, :])
                    
                is_true_action = (gt_actions[jmax, aid] == 1)
                    
                if ovmax < 0.5: # person mis-loc
                    fp1 += 1
                    res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [1]]))
                elif 0 <= ov_max_all_boxes < 0.5:
                    if is_true_action == True and ov_max_j_objs == -1.0: # occlusion
                        fp6 += 1
                        res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [6]]))
                    else: # obj misloc
                        fp2 += 1
                        res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [2]]))
                elif 0 <= ov_max_all_relboxes < 0.5: # misgrouping, obj not attend any rel
                    fp3 += 1
                    
#                     pred_roles[valid[j], 5 * aid + 4, rid] = 0
                    
                    res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [3]]))
                elif ov_max_j_objs < 0.5: # misgrouping, obj attend rel but not such person
                    fp4 += 1
                    
#                     res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [4]]))
                    
                    pred_roles[valid[j], 5 * aid + 4, rid] = 0
                elif ov_role < 0.5: # prediction error
                    fp5 += 1
#                     pred_roles[valid[j], 5 * aid + 4, rid] = 0
                    res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [5]]))
                elif (ovmax >= 0.5) & (ov_role >= 0.5):  # true positive
                    if not covered[jmax]:
                        tp += 1
                        res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [0]]))
                        covered[jmax] = True
                    else:   # triplet duplicate
                        fp7 += 1
                        res.append(np.concatenate([pred_box, role_boxes[j, :], [aid, rid, agent_scores[j]], [7]]))
#     set_zero_ret[image_id] = {'agents': pred_agents, 'roles':pred_roles}
    ret = {}
    ret['im_id'] = image_id
    ret['gt_classes'] = vcocodb[i]['gt_classes']
    ret['gt_boxes'] = vcocodb[i]['boxes']
    ret['gt_actions'] = vcocodb[i]['gt_actions']
    ret['gt_role_inds'] = vcocodb[i]['gt_role_id']
    ret['pred_triplets'] = np.array(res)
    all_res.append(ret)
print(fp1,fp2,fp3,fp4,fp5,fp6,tp)
print(count)

46368 36549 62905 9755 47879 201 9029
212688


In [33]:
print(fp1,fp2,fp3,fp4,fp5,fp6)
fps = fp1+fp2+fp3+fp4+fp5+fp6
gt_num = 12423
print('gt triplet num:', gt_num)
print('tp num:', tp)
print('fp num:', fps)
print('recall:', tp/gt_num)
print('person misloc:', fp1/fps)
print('obj misloc:', fp2/fps)
print('misgrouping, obj not attend any rel:', fp3/fps)
print('misgrouping, obj attend rel but not such person:', fp4/fps)
print('prediction wrong:', fp5/fps)
print('occlusion:', fp6/fps)
# print('duplicate:', fp7/fps)

46368 36549 62905 9755 47879 201
gt triplet num: 12423
tp num: 9029
fp num: 203657
recall: 0.7267970699508975
person misloc: 0.2276769273828054
obj misloc: 0.17946350972468414
misgrouping, obj not attend any rel: 0.3088771807499865
misgrouping, obj attend rel but not such person: 0.04789916379009806
prediction wrong: 0.23509626479816553
occlusion: 0.0009869535542603495


In [28]:
json_dataset.VCOCO._do_role_eval(vcocodb, set_zero_ret, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 46.02 (#pos = 3608)
              sit-instr: AP = 28.86 (#pos = 1916)
             ride-instr: AP = 68.40 (#pos = 556)
               look-obj: AP = 49.11 (#pos = 3347)
              hit-instr: AP = 77.01 (#pos = 349)
                hit-obj: AP = 52.69 (#pos = 349)
                eat-obj: AP = 44.36 (#pos = 521)
              eat-instr: AP = 6.05 (#pos = 521)
             jump-instr: AP = 53.36 (#pos = 635)
              lay-instr: AP = 30.53 (#pos = 387)
    talk_on_phone-instr: AP = 55.40 (#pos = 285)
              carry-obj: AP = 43.54 (#pos = 472)
              throw-obj: AP = 68.80 (#pos = 244)
              catch-obj: AP = 61.72 (#pos = 246)
              cut-instr: AP = 38.24 (#pos = 269)
                cut-obj: AP = 44.30 (#pos = 269)
 work_on_computer-instr: AP = 64.79 (#pos = 410)
              ski-instr: AP = 52.10 (#pos = 424)
             surf-instr: AP = 81.12 (#pos = 486)
       skateboard-

In [50]:
# file = './Outputs/e2e_interact_net_R-50-FPN_1x/Nov14-09-23-26_wanbo_node33_step/test/hoi_vcoco_triplet_results.pkl'
# save_file = './Outputs/e2e_interact_net_R-50-FPN_1x/Nov14-09-23-26_wanbo_node33_step/images/'
file = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_triplet_results_ican.pkl'
save_file = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/images/'
# file = './ican/hoi_vcoco_triplet_results_ican.pkl'
# save_file = './ican/images/'
if not os.path.exists(save_file):
    os.mkdir(save_file)
f = open(file, 'rb')
all_res = pickle.load(f, encoding='latin1')

In [51]:
no_gt_image_num = 0
pred_no_human_num = 0
fp1 = fp2 = fp3 = fp4 = fp5 = fp6 = tp = 0
gt_num = 0
count = 0
for i, ele in enumerate(all_res):
    image_id = ele['im_id']
    gt_classes = ele['gt_classes']
    gt_boxes = ele['gt_boxes']
    gt_actions = ele['gt_actions']
    pred_triplets = ele['pred_triplets']
    gt_role_inds = ele['gt_role_inds']

    gt_human_inds, gt_act_inds, gt_r_ids = np.where(gt_role_inds>-1)
    gt_obj_inds = gt_role_inds[gt_human_inds, gt_act_inds, gt_r_ids]
    gt_human_box = gt_boxes[gt_human_inds].astype(np.int)
    gt_obj_boxes = gt_boxes[gt_obj_inds].astype(np.int)
    
    gt_num += len(gt_human_inds)
    if len(gt_human_inds) == 0:
        no_gt_image_num += 1
        continue
    if pred_triplets.shape[0] == 0:
        pred_no_human_num += 1
        continue
    
    
    scores = pred_triplets[:, -2]
    idx = scores.argsort()[::-1]
#     triplets = pred_triplets[idx[:5]]
#     sorted_score = scores[idx[:5]]
    triplets = pred_triplets[idx]
    sorted_score = scores[idx]

    for m, t in enumerate(triplets):
#         if sorted_score[m] < 0.1:
#             continue
        h_box = tuple(t[:4].astype(np.int))
        o_box = tuple(t[4:8].astype(np.int))
        aid, rid, score, error_type = t[-4:]
        text = action_names[aid.astype(np.int)] + '_' + role_names[rid.astype(np.int)]
        if error_type == 0: # tp
            tp += 1
        elif error_type == 1: # person misloc
            fp1 += 1
        elif error_type == 2:  # human loc is right but obj misloc
            fp2 += 1
        elif error_type == 3: # the target object is fg but not in target triplet; misgrouping
            fp3 += 1
        elif error_type == 4: # person and obj loc is right but predict action wrong; prediction wrong
            fp4 += 1
        elif error_type == 5: # occlusion
            fp5 += 1
        elif error_type == 6: # triplet duplicate
            fp6 += 1

In [18]:
print(fp1,fp2,fp3,fp4,fp5)
fps = fp1+fp2+fp3+fp4+fp5
print('gt triplet num:', gt_num)
print('tp num:', tp)
print('fp num:', fps)
print('recall:', tp/gt_num)
print('person misloc:', fp1/fps)
print('obj misloc:', fp2/fps)
print('misgrouping:', fp3/fps)
print('prediction wrong:', fp4/fps)
print('occlusion:', fp5/fps)

4726 13431 25522 3157 32455
gt triplet num: 12423
tp num: 8951
fp num: 79291
recall: 0.7205183933027449
person misloc: 0.05960323365829665
obj misloc: 0.16938870741950537
misgrouping: 0.32187764058972645
prediction wrong: 0.03981536366044065
occlusion: 0.4093150546720309


In [7]:
vcoco_anns = '../data/coco/vcoco/vcoco/vcoco_test.json'
with open(vcoco_anns, 'r') as f:
    vsrl_data = json.load(f)
action_names = []
for j in vsrl_data:
    action_names.append(j['action_name'])

role_names = ['obj','instr']
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
yellow = (255, 255, 0)
cyan = (0, 255, 255)
# font = cv2.FONT_HERSHEY_SIMPLEX
font = cv2.FONT_HERSHEY_PLAIN

In [8]:
no_gt_image_num = 0
pred_no_human_num = 0
fp1 = fp2 = fp3 = fp4 = fp5 = fp6 = tp = 0
save_file = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/images1/'
for i, ele in enumerate(all_res):
    if i%50==0:
        print(i)
    image_id = ele['im_id']
    gt_classes = ele['gt_classes']
    gt_boxes = ele['gt_boxes']
    gt_actions = ele['gt_actions']
    pred_triplets = ele['pred_triplets']
    gt_role_inds = ele['gt_role_inds']


    gt_human_inds, gt_act_inds, gt_r_ids = np.where(gt_role_inds>-1)
    gt_obj_inds = gt_role_inds[gt_human_inds, gt_act_inds, gt_r_ids]
    gt_human_box = gt_boxes[gt_human_inds].astype(np.int)
    gt_obj_boxes = gt_boxes[gt_obj_inds].astype(np.int)
    
    if len(gt_human_inds) == 0:
        no_gt_image_num += 1
        continue
    if pred_triplets.shape[0] == 0:
        pred_no_human_num += 1
        continue
        
    im_file = '../data/coco/images/val2014/COCO_val2014_' + (str(image_id)).zfill(12) + '.jpg'
    save_dir = os.path.join(save_file, 'COCO_val2014_' + (str(image_id)).zfill(12))
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        
    for n in range(len(gt_human_inds)):
        img = miscp.imread(im_file)
        gt_h_box = tuple(gt_human_box[n])
        gt_o_box = tuple(gt_obj_boxes[n])
        text = action_names[gt_act_inds[n]] + '_' + role_names[gt_r_ids[n]]
        cv2.putText(img, text, gt_o_box[:2], font, 1.2, green, thickness=2)
        cv2.rectangle(img, gt_h_box[:2], gt_h_box[2:], green, 2)
        cv2.rectangle(img, gt_o_box[:2], gt_o_box[2:], green, 2)
        miscp.imsave(os.path.join(save_dir, 'gt_{}.jpg'.format(n)), img)
    
    scores = pred_triplets[:, -2]
    idx = scores.argsort()[::-1]
    triplets = pred_triplets[idx[:10]]
    sorted_score = scores[idx[:10]]

    for m, t in enumerate(triplets):
        if sorted_score[m] < 0.1:
            continue
        img = miscp.imread(im_file)
        h_box = tuple(t[:4].astype(np.int))
        o_box = tuple(t[4:8].astype(np.int))
        aid, rid, score, error_type = t[-4:]
#         ipdb.set_trace()
        score_text = "%.3f"%(sorted_score[m])
        text = action_names[aid.astype(np.int)] + '_' + role_names[rid.astype(np.int)]
        if error_type == 0: # tp
            tp += 1
            h_color = green
            o_color = green
            a_color = green
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
            cv2.putText(img, text, o_box[:2], font, 1.2, a_color, thickness=2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 1: # person misloc
            fp1 += 1
            continue
            h_color = red
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 2: # human loc is right but obj misloc
            fp2 += 1
            continue
            h_color = green
            o_color = red
            a_color = cyan
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
            cv2.putText(img, text, o_box[:2], font, 1.2, a_color, thickness=2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 3:  # the target object is fg but not in target triplet; misgrouping
            fp3 += 1
            continue
            h_color = blue
            o_color = blue
            a_color = cyan
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
            cv2.putText(img, text, (h_box[0], h_box[1]+20), font, 1.2, a_color, thickness=2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 4:  # the target object is fg but not in target triplet; misgrouping
            fp4 += 1
            continue
            h_color = yellow
            o_color = yellow
            a_color = cyan
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
            cv2.putText(img, text, (h_box[0], h_box[1]+20), font, 1.2, a_color, thickness=2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 5:  # person and obj loc is right but predict action wrong; prediction wrong
            fp5 += 1
            h_color = green
            o_color = green
            a_color = red
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
            cv2.putText(img, text, (h_box[0], h_box[1]+20), font, 1.2, a_color, thickness=2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 6: # occlusion
            fp6 += 1
            continue
            h_color = yellow
            a_color = yellow
            o_color = yellow
            cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
            cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
            cv2.putText(img, text, (h_box[0], h_box[1]+20), font, 1.2, a_color, thickness=2)
            cv2.putText(img, score_text, (20, 20), font, 1.2, green, thickness=2)
        elif error_type == 7: # triplet duplicate
            fp7 += 1
            continue
#             h_color = cyan
#             o_color = cyan
#             cv2.rectangle(img, h_box[:2], h_box[2:], h_color, 2)
#             cv2.rectangle(img, o_box[:2], o_box[2:], o_color, 2)
        miscp.imsave(os.path.join(save_dir, str(m)+'.jpg'), img)

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:129: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050
4100
4150
4200
4250
4300
4350
4400
4450
4500


In [37]:
print(no_gt_image_num, pred_no_human_num)

0 24


In [38]:
target_path = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/images_filter/'
dirs = os.listdir(save_file)
count = 0
for diri in dirs:
    child = os.path.join('%s/%s' % (save_file, diri))
    flag = 0
    path_dir1 = os.listdir(child)
    for img_name in path_dir1:
        if 'gt' not in img_name:
            flag += 1
    if flag > 0:
        os.system('cp -r {} {}'.format(child, target_path))
        count += 1
print(count)

2402


In [47]:
det_name1 = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_test_results.pkl'
det_name2 = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_test_results_no_misgrouping.pkl'
dets1 = pickle.load(open(det_name1, 'rb') , encoding='latin1')
dets2 = pickle.load(open(det_name2, 'rb') , encoding='latin1')
json_dataset.VCOCO._do_role_eval(vcocodb, dets1, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)
json_dataset.VCOCO._do_role_eval(vcocodb, dets2, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 38.46 (#pos = 3608)
              sit-instr: AP = 24.80 (#pos = 1916)
             ride-instr: AP = 63.56 (#pos = 556)
               look-obj: AP = 36.00 (#pos = 3347)
              hit-instr: AP = 70.87 (#pos = 349)
                hit-obj: AP = 44.54 (#pos = 349)
                eat-obj: AP = 39.03 (#pos = 521)
              eat-instr: AP = 4.14 (#pos = 521)
             jump-instr: AP = 47.76 (#pos = 635)
              lay-instr: AP = 23.86 (#pos = 387)
    talk_on_phone-instr: AP = 52.23 (#pos = 285)
              carry-obj: AP = 34.01 (#pos = 472)
              throw-obj: AP = 44.48 (#pos = 244)
              catch-obj: AP = 45.45 (#pos = 246)
              cut-instr: AP = 34.09 (#pos = 269)
                cut-obj: AP = 37.75 (#pos = 269)
 work_on_computer-instr: AP = 58.71 (#pos = 410)
              ski-instr: AP = 46.44 (#pos = 424)
             surf-instr: AP = 78.65 (#pos = 486)
       skateboard-

In [58]:
det_name3 = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_test_results_no_actmiscls.pkl'
dets3 = pickle.load(open(det_name3, 'rb') , encoding='latin1')
json_dataset.VCOCO._do_role_eval(vcocodb, dets3, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 41.58 (#pos = 3608)
              sit-instr: AP = 25.48 (#pos = 1916)
             ride-instr: AP = 64.77 (#pos = 556)
               look-obj: AP = 42.21 (#pos = 3347)
              hit-instr: AP = 74.56 (#pos = 349)
                hit-obj: AP = 47.55 (#pos = 349)
                eat-obj: AP = 40.51 (#pos = 521)
              eat-instr: AP = 5.07 (#pos = 521)
             jump-instr: AP = 52.40 (#pos = 635)
              lay-instr: AP = 28.97 (#pos = 387)
    talk_on_phone-instr: AP = 54.59 (#pos = 285)
              carry-obj: AP = 37.51 (#pos = 472)
              throw-obj: AP = 62.98 (#pos = 244)
              catch-obj: AP = 56.92 (#pos = 246)
              cut-instr: AP = 36.05 (#pos = 269)
                cut-obj: AP = 40.14 (#pos = 269)
 work_on_computer-instr: AP = 59.59 (#pos = 410)
              ski-instr: AP = 48.20 (#pos = 424)
             surf-instr: AP = 80.30 (#pos = 486)
       skateboard-

In [59]:
det_name4 = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_test_results_no_objmisloc.pkl'
dets4 = pickle.load(open(det_name3, 'rb') , encoding='latin1')
json_dataset.VCOCO._do_role_eval(vcocodb, dets4, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 42.99 (#pos = 3608)
              sit-instr: AP = 29.53 (#pos = 1916)
             ride-instr: AP = 66.14 (#pos = 556)
               look-obj: AP = 39.26 (#pos = 3347)
              hit-instr: AP = 72.15 (#pos = 349)
                hit-obj: AP = 47.54 (#pos = 349)
                eat-obj: AP = 43.16 (#pos = 521)
              eat-instr: AP = 5.42 (#pos = 521)
             jump-instr: AP = 48.74 (#pos = 635)
              lay-instr: AP = 26.07 (#pos = 387)
    talk_on_phone-instr: AP = 56.81 (#pos = 285)
              carry-obj: AP = 40.13 (#pos = 472)
              throw-obj: AP = 45.78 (#pos = 244)
              catch-obj: AP = 46.49 (#pos = 246)
              cut-instr: AP = 40.58 (#pos = 269)
                cut-obj: AP = 42.35 (#pos = 269)
 work_on_computer-instr: AP = 62.85 (#pos = 410)
              ski-instr: AP = 53.20 (#pos = 424)
             surf-instr: AP = 80.18 (#pos = 486)
       skateboard-

In [65]:
# upper bound: 58.52
det_name5 = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_test_results_no_fp.pkl'
dets5 = pickle.load(open(det_name5, 'rb') , encoding='latin1')
json_dataset.VCOCO._do_role_eval(vcocodb, dets5, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 50.55 (#pos = 3608)
              sit-instr: AP = 33.47 (#pos = 1916)
             ride-instr: AP = 70.97 (#pos = 556)
               look-obj: AP = 52.39 (#pos = 3347)
              hit-instr: AP = 78.18 (#pos = 349)
                hit-obj: AP = 54.49 (#pos = 349)
                eat-obj: AP = 48.44 (#pos = 521)
              eat-instr: AP = 8.87 (#pos = 521)
             jump-instr: AP = 55.30 (#pos = 635)
              lay-instr: AP = 36.89 (#pos = 387)
    talk_on_phone-instr: AP = 60.61 (#pos = 285)
              carry-obj: AP = 52.46 (#pos = 472)
              throw-obj: AP = 72.78 (#pos = 244)
              catch-obj: AP = 66.19 (#pos = 246)
              cut-instr: AP = 47.82 (#pos = 269)
                cut-obj: AP = 50.99 (#pos = 269)
 work_on_computer-instr: AP = 69.04 (#pos = 410)
              ski-instr: AP = 60.16 (#pos = 424)
             surf-instr: AP = 83.20 (#pos = 486)
       skateboard-

In [66]:
det_name6 = '../Outputs/e2e_interact_net_R-50-FPN_1x/Dec10-15-24-10_wanbo_node23_step/test/hoi_vcoco_test_results_no_34error.pkl'
dets6 = pickle.load(open(det_name6, 'rb') , encoding='latin1')
json_dataset.VCOCO._do_role_eval(vcocodb, dets6, ovr_thresh=0.5, eval_type='scenario_1', plot_save_path=None)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 42.48 (#pos = 3608)
              sit-instr: AP = 26.30 (#pos = 1916)
             ride-instr: AP = 66.28 (#pos = 556)
               look-obj: AP = 44.49 (#pos = 3347)
              hit-instr: AP = 75.36 (#pos = 349)
                hit-obj: AP = 49.22 (#pos = 349)
                eat-obj: AP = 41.73 (#pos = 521)
              eat-instr: AP = 5.17 (#pos = 521)
             jump-instr: AP = 52.83 (#pos = 635)
              lay-instr: AP = 29.12 (#pos = 387)
    talk_on_phone-instr: AP = 54.89 (#pos = 285)
              carry-obj: AP = 38.09 (#pos = 472)
              throw-obj: AP = 66.79 (#pos = 244)
              catch-obj: AP = 59.62 (#pos = 246)
              cut-instr: AP = 36.76 (#pos = 269)
                cut-obj: AP = 41.72 (#pos = 269)
 work_on_computer-instr: AP = 61.01 (#pos = 410)
              ski-instr: AP = 49.24 (#pos = 424)
             surf-instr: AP = 80.89 (#pos = 486)
       skateboard-